Hamiltonians From Scratch*
===========================
*Its not really from scratch because I use the FermionicOp class to make it work

This is an alternative to the `qiskis_nature` Fermi-Hubbard Hamiltonian. I get the same ground state energies but a different ground state vector. To use this hamiltonian import it from the .py file [here](./defining_the_hamiltonian.py) 

In [2]:
from qiskit import *
from qiskit.quantum_info.operators import SparsePauliOp
from qiskit.quantum_info.operators import Pauli
from qiskit.quantum_info import Operator
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper

import numpy as np
from functools import *
import scipy.linalg as LA

In [3]:
# Fermionic operators that we need
creation_ferms = [FermionicOp({f"+_{i}":1}) for i in range(0,12)]
annihilation_ferms = [FermionicOp({f"-_{i}":1}) for i in range(0,12)]
number_ferms = [creation_ferms[i] @ annihilation_ferms[i] for i in range(0, 12)]

In [4]:

verticies = 6
spins = [0,1]
edges = [(0,1),(0,2),(0,3),(0,4),
         (5,1),(5,2),(5,3),(5,4),
         (1,2),(2,3),(3,4),(4,1)]

myTensor = lambda a, b: (a)^(b)

def swap_electron(edge:tuple[int, int], s:int, verticies:int) -> FermionicOp:
    """Swaps the electron in spin state s in vertex i and vertex j."""
    (i, j) = edge
    swap_1 = creation_ferms[i+verticies*s] @ annihilation_ferms[j+verticies*s]
    swap_2 = creation_ferms[j+verticies*s] @ annihilation_ferms[i+verticies*s]
    return swap_1 + swap_2

def number_up_down(vertex:int , verticies:int) -> FermionicOp:
    return number_ferms[vertex + verticies] @ number_ferms[vertex]

total_swap_operator:FermionicOp = reduce(lambda a, b: (a) + (b), 
                             [swap_electron(edge, s, verticies) for edge in edges for s in spins])
total_number_operator:FermionicOp = reduce(lambda a, b: (a) + (b), 
                               [number_up_down(i, verticies) for i in range(0, verticies)])

t = 1
u = 1
hamiltonian_ferm = -t*total_swap_operator + u*total_number_operator
hamiltonian = JordanWignerMapper().map(hamiltonian_ferm)


In [5]:
print("Dims in hamiltonian:", hamiltonian.dim)
print("hamiltonian hermitian?:", Operator(hamiltonian) == Operator(hamiltonian).adjoint())

Dims in hamiltonian: (4096, 4096)
hamiltonian hermitian?: True


In [6]:
H_matrix = hamiltonian.to_matrix()
eigs = LA.eigh(H_matrix)


In [7]:
eigenvals, eigenvectors = eigs

In [2]:
print(eigenvals[26:35])
print(eigenvals[-10:])


NameError: name 'eigenvals' is not defined

In [9]:
ground_state = [(x, f"{i:012b}") for i,x in enumerate(eigenvectors[:, 0]) if abs(x) > 1e-8]
(ground_state)

[(np.complex128(-0.15253406813364298+0j), '000001000001'),
 (np.complex128(-0.16869036372022123+0j), '000001000010'),
 (np.complex128(-0.16869036372022123+0j), '000001000100'),
 (np.complex128(-0.16869036372022123+0j), '000001001000'),
 (np.complex128(-0.1686903637202212+0j), '000001010000'),
 (np.complex128(-0.17197171126969077+0j), '000001100000'),
 (np.complex128(-0.16869036372022123+0j), '000010000001'),
 (np.complex128(-0.15253406813364292+0j), '000010000010'),
 (np.complex128(-0.1686903637202212+0j), '000010000100'),
 (np.complex128(-0.1719717112696908+0j), '000010001000'),
 (np.complex128(-0.1686903637202212+0j), '000010010000'),
 (np.complex128(-0.16869036372022125+0j), '000010100000'),
 (np.complex128(-0.16869036372022123-0j), '000100000001'),
 (np.complex128(-0.16869036372022114-0j), '000100000010'),
 (np.complex128(-0.1525340681336429-0j), '000100000100'),
 (np.complex128(-0.1686903637202212+0j), '000100001000'),
 (np.complex128(-0.1719717112696908+0j), '000100010000'),
 (np

In [10]:
(c0, _) = ground_state[0] # same node
c0_elements = [(c, i) for c,i in ground_state if c0-1e-5 < c < c0+1e-5]

(c1, _) = ground_state[1] # adjacemt nodes
c1_elements = [(c, i) for c,i in ground_state if c1-1e-5 < c < c1+1e-5]

(c2, _) = ground_state[19] # oposite nodes
c2_elements = [(c, i) for c,i in ground_state if c2-1e-5 < c < c2+1e-5]

print(f"c0:{c0}")
print(f"c1:{c1}")
print(f"c2:{c2}")

ground_state[1]

c0:(-0.15253406813364298+0j)
c1:(-0.16869036372022123+0j)
c2:(-0.17197171126969088+0j)


(np.complex128(-0.16869036372022123+0j), '000001000010')

In [11]:
def num_of_ones(s:str) -> int:
    return len(list(filter(lambda x: x=='1', s)))

len(list(filter(lambda x: x==4, [num_of_ones(s) for c,s in ground_state])))

0